In [1]:
import pandas as pd
import datetime
import numpy as np
import sklearn
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model
from sklearn.metrics import *
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

C:\Users\Akash\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Akash\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
col_list=['Page_popularity','Page_visited_no_of_times','Page_talking_about','Page_category','c1','c2','c3','c4','c5','c6','c7','c8',
         'c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24','c25','CC1','CC2','CC3','CC4','CC5',
         'Base_time','Post_length_char_count','Post_share_count','Post_promoted','Time_target','Sunday_post','Monday_post',
         'Tuesday_post','Wednesday_post','Thrusday_post','Friday_post','Saturday_post','Sunday_base','Monday_base','Tuesday_base','Wednesday_base',
         'Thrusday_base','Friday_base','Saturday_base','Target_variable']
d1=pd.read_csv("Dataset/Training/Features_Variant_1.csv")
d1.columns=col_list
d2=pd.read_csv("Dataset/Training/Features_Variant_2.csv")
d2.columns=col_list
d3=pd.read_csv("Dataset/Training/Features_Variant_3.csv")
d3.columns=col_list
d4=pd.read_csv("Dataset/Training/Features_Variant_4.csv")
d4.columns=col_list
d5=pd.read_csv("Dataset/Training/Features_Variant_5.csv")
d5.columns=col_list

In [3]:
frames_main = [d1 , d2 , d3 , d4 , d5]
df = pd.concat(frames_main)

In [5]:
df.shape
df.head()

,Page_popularity,Page_visited_no_of_times,Page_talking_about,Page_category,c1,c2,c3,c4,c5,c6,...,Friday_post,Saturday_post,Sunday_base,Monday_base,Tuesday_base,Wednesday_base,Thrusday_base,Friday_base,Saturday_base,Target_variable
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,1,0,0,0,0,0,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,1,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train.iloc[:,1:53]
y_train=df_train['Target_variable']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,1:53]
y_test=df_test['Target_variable']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

In [8]:
#Random Hyperparameter Grid
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 11)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 19, 28, 37, 46, 55, 64, 73, 82, 91, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
rdf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rdf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train_sc, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 24.0min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 202.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 1181.8min finished


In [10]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 90,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 82}

In [14]:
model=rf_random.best_estimator_
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=90,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=82, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [13]:
y_train_pred=model.predict(x_train_sc)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))

R2   : 0.964659644404
MAE  : 1.16910523539
RMSE : 6.47673339965


In [15]:
y_test_pred=model.predict(x_test_sc)
print("R2   :",r2_score(y_test,y_test_pred))
print("MAE  :",mean_absolute_error(y_test,y_test_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_test_pred)))

R2   : 0.726247308112
MAE  : 3.3757742432
RMSE : 18.0430651155


In [16]:
feature_list = list(x_train.columns)
importances = list(model.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: CC2                  Importance: 0.28
Variable: Base_time_something  Importance: 0.21
Variable: Post_share_count     Importance: 0.11
Variable: c3                   Importance: 0.03
Variable: CC4                  Importance: 0.03
Variable: Page_talking_about   Importance: 0.02
Variable: c13                  Importance: 0.02
Variable: c15                  Importance: 0.02
Variable: c18                  Importance: 0.02
Variable: CC1                  Importance: 0.02
Variable: CC5                  Importance: 0.02
Variable: Post_length_char_count Importance: 0.02
Variable: Page_visited_no_of_times Importance: 0.01
Variable: Page_category        Importance: 0.01
Variable: c2                   Importance: 0.01
Variable: c4                   Importance: 0.01
Variable: c5                   Importance: 0.01
Variable: c8                   Importance: 0.01
Variable: c9                   Importance: 0.01
Variable: c10                  Importance: 0.01
Variable: c12                  Imp

Building Random Forest Model by droping least singnificant features

In [32]:
drop_col_list=['Saturday_base','Friday_base','Thrusday_base','Wednesday_base','Tuesday_base','Monday_base','Sunday_base','Saturday_post',
              'Friday_post','Thrusday_post','Wednesday_post','Tuesday_post','Monday_post','Sunday_post','Post_promoted','CC3',
              'c16','c11','c6','c1'
               ,'c24','c23','c22','c21','c20','c17','c15','c14','c12','c10','c7','c5','c2','Page_category','Page_talking_about',
               'Page_visited_no_of_times','c25','c19','c13','c9','c4','Time_target']
x_train=df_train.iloc[:,1:53]
x_train.drop(drop_col_list,axis=1,inplace=True)
y_train=df_train['Target_variable']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,1:53]
x_test.drop(drop_col_list,axis=1,inplace=True)
y_test=df_test['Target_variable']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

Random Forest Model After using tuned  hyperparameters which we got using random grid search

In [33]:
rf=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=90,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=82, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
rf.fit(x_train_sc, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=90,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=82, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

Random Forest on Training dataset

In [34]:
y_train_pred=rf.predict(x_train_sc)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))

R2   : 0.962281332043
MAE  : 1.2092783175
RMSE : 6.69111873662


Random Forest on Testing dataset

In [35]:
y_test_pred=rf.predict(x_test_sc)
print("R2   :",r2_score(y_test,y_test_pred))
print("MAE  :",mean_absolute_error(y_test,y_test_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_test_pred)))

R2   : 0.711138439768
MAE  : 3.45171654852
RMSE : 18.5342916891


Importance of features for Random Forest Models

In [36]:
feature_list = list(x_train.columns)
importances = list(rf.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Base_time_something  Importance: 0.23
Variable: CC2                  Importance: 0.14
Variable: CC5                  Importance: 0.13
Variable: Post_share_count     Importance: 0.12
Variable: c3                   Importance: 0.08
Variable: c8                   Importance: 0.07
Variable: c18                  Importance: 0.07
Variable: CC1                  Importance: 0.05
Variable: CC4                  Importance: 0.05
Variable: Post_length_char_count Importance: 0.05
